<a href="figures/Nexus_logo_800px.png" target="_blank"><img src="figures/Nexus_logo_800px.png" 
width="150" border="10" /></a>

# Blue Brain Nexus - A knowledge graph for data-driven science

## 5 - Exercises Part 2

### 5-0 Import of relevant Pyxus classes and client setup

In [3]:
import requests
import yaml
from pyxus.client import NexusClient
from pyxus.resources.entity import Organization
from pyxus.resources.entity import Domain
from pyxus.resources.entity import Schema
from pyxus.resources.entity import Instance

Grab a token here (use your Collab credentials to log in): https://bbp-nexus.epfl.ch/staging/v0/oauth2/authorize

Copy the token (the string of numbers and letters between the quotation marks) and paste it as the value of the 'token' variable below:

In [40]:
# token = "<PASTE YOUR TOKEN HERE>"
token = ""

In [41]:
client = NexusClient(scheme="https", host="bbp-nexus.epfl.ch", prefix="staging/v0", alternative_namespace="https://bbp-nexus.epfl.ch", token=token)

In [ ]:
print(client.organizations)
print(client.domains)
print(client.schemas)
print(client.instances)

### 5-1 Create a custom organization on Nexus using Pyxus

Input a custom name and description for your organization below:

In [32]:
organization_name = "<PUT YOUR ORGANIZATION NAME HERE>"
organization_description = "PUT YOUR ORGANIZATION DESCRIPTION HERE"

org1
dd


With the provided organization name and description, you can create your organization on Nexus:

In [ ]:
your_organization = Organization.create_new(organization_name, organization_description) 
client.organizations.create(your_organization)

You can access your organization here:

In [ ]:
your_organization = client.organizations.read(organization_name)
print(your_organization.data["@id"])

### 5-2 Create a custom domain on Nexus using Pyxus

Input a custom name and decription for your domain below:

In [33]:
domain_name = "<PUT YOUR DOMAIN NAME HERE>"
domain_description = "PUT YOUR DOMAIN DESCRIPTION HERE"

experiment
ww


With the provided domain name and description, you can create your domain on Nexus:

In [ ]:
your_domain = Domain.create_new(organization_name, domain_name, domain_description) 
client.domains.create(your_domain)

You can access your domain here:

In [ ]:
your_domain = client.domains.read(organization_name, domain_name)
print(your_domain.data["@id"])

### 5-3 Secure the custom organization

To restrict access to your organization, you can set specific access rights

Get your user reference and realm here:

In [ ]:
response = requests.get("https://bbp-nexus.epfl.ch/staging/v0/oauth2/user", headers={"Authorization": "Bearer {}".format(token)})
if response.status_code < 400:
    response_text = yaml.load(response.text)["identities"]
    for identity in response_text:
        if identity["@type"] == "UserRef":
            user_reference = identity["sub"]
            realm = identity["realm"]
else:
     print(response.status_code)

print(user_reference)
print(realm) 

The following payload defines the permissions you want to set for your organization

In [ ]:
acls = {
    "acl": [
        {
            "permissions": [
            "read",
            "write",
            "own"
            ],
            "identity": {
            "realm": realm,
            "sub": user_reference,
            "@type": "UserRef"
            }
        }
    ]
}

Use the defined acls to set permissions on your organization:

In [ ]:
response = requests.put(your_organization.data["@id"], json=acls)
if response.status_code >= 400:
    response_text = yaml.load(response.text)
    print(response_text.get('code'))
else:
    print(response.status_code)

### 5-4 Create and publish schemas for Subject, Neuron and Dataset

Create the Schemas for Subject, Neuron and Dataset:

In [37]:
subject_schema = {
  "@context": [
    "https://bbp-nexus.epfl.ch/staging/v0/contexts/neurosciencegraph/core/schema/v0.1.0"
  ],
  "@type": "nxv:Schema",
  "shapes": [
    {
      "@id": "https://bbp-nexus.epfl.ch/staging/v0/schemas/neurosciencegraph/experiment/subject/v0.1.0/shapes/SubjectShape",
      "@type": "sh:NodeShape",
      "label": "Subject shape definition",
      "comment": "Subject used in experiment.",
      "targetClass": "nsg:Subject",
      "nodeKind": "sh:BlankNodeOrIRI",
      "property": [
        {
          "path": "schema:name",
          "name": "Name",
          "description": "The name of the subject",
          "datatype": "xsd:string",
          "maxCount": 1
        },
        {
          "path": "nsg:species",
          "name": "Species",
          "description": "The species of the subject",
          "datatype": "xsd:string",
          "minCount": 1,
          "maxCount": 1
        },
        {
          "path": "nsg:sex",
          "name": "Sex",
          "description": "The sex of the subject",
          "datatype": "xsd:string",
          "maxCount": 1
        },
        {
          "path": "nsg:age",
          "name": "Age",
          "description": "The age of the subject.",
          "node": "https://bbp-nexus.epfl.ch/staging/v0/schemas/neurosciencegraph/experiment/subject/v0.1.0/AgeShape",
          "maxCount": 1
        },
        {
          "path": "nsg:weight",
          "name": "Weight",
          "description": "The weight of the subject.",
          "node": "https://bbp-nexus.epfl.ch/staging/v0/schemas/neurosciencegraph/experiment/subject/v0.1.0/QuantitativeValueShape",
          "maxCount": 1
        }
      ]
    },
    {
      "@id": "https://bbp-nexus.epfl.ch/staging/v0/schemas/neurosciencegraph/experiment/subject/v0.1.0/AgeShape",
      "@type": "sh:NodeShape",
      "property": [
        {
          "path": "nsg:period",
          "name": "Period",
          "in": [
            "Pre-natal",
            "Post-natal"
          ],
          "minCount": 1,
          "maxCount": 1
        },
        {
          "path": "schema:value",
          "name": "Age value",
          "node": "https://bbp-nexus.epfl.ch/staging/v0/schemas/neurosciencegraph/experiment/subject/v0.1.0/QuantitativeValueShape",
          "minCount": 1,
          "maxCount": 1
        }
      ]
    },
    {
      "@id": "https://bbp-nexus.epfl.ch/staging/v0/schemas/neurosciencegraph/experiment/subject/v0.1.0/QuantitativeValueShape",
      "@type": "sh:NodeShape",
      "property": [
        {
          "path": "schema:value",
          "name": "Value",
          "datatype": "xsd:string",
          "maxCount": 1,
          "minCount": 1
        },
        {
          "path": "schema:unitText",
          "name": "Unit",
          "datatype": "xsd:string",
          "maxCount": 1,
          "minCount": 1
        }

      ]
    }
  ]
}

In [ ]:
neuron_schema = {}

In [ ]:
dataset_schema = {}

Repeat the following steps for the Subject, Neuron and dataset schemas:

In [38]:
schema_name = "subject"
schema_version = "v0.1.0"
content = subject_schema

In [ ]:
schema = Schema.create_new(organization=organization_name, domain=domain_name, schema='subject',version='v0.1.0', content=content)
client.schemas.create(schema)

In [131]:
schema = client.schemas.read(organization=organization_name, domain=domain_name, schema=schema_name, version=schema_version)
print(subject.data["@id"])

https://bbp-nexus.epfl.ch/staging/v0/schemas/org1/experiment/subject/v0.1.0


Publish the schema you have just created:

In [133]:
client.schemas.publish(entity=schema,publish=True)

### 5-5 Create instances with prepared payloads

In [66]:
subject_data = {
"@context": [
  "https://bbp-nexus.epfl.ch/staging/v0/contexts/neurosciencegraph/core/data/v0.1.1"
  ],
  "@type": [
    "nsg:Subject"
    ],
  "name": "Name",
  "species": "Rat",
  "strain": "",
  "sex": "",
  "age": {
    "period": "Post-natal",
    "value": {
      "unitText": "",
      "value": ""
    }
  },
  "weight": {
    "unitText": "",
    "value": ""
  }
}

In [ ]:
neuron_data = {}

In [ ]:
dataset_data = {}

In [58]:
instance = Instance.create_new(organization=organization_name, domain=domain_name, schema=schema_name, version=schema_version, content=subject_data)
client.instances.create(instance)

Retrieve all the instances you have created

In [68]:
search_result = client.instances.list_by_schema(organization=organization_name, domain=domain_name, schema=schema_name, version=schema_version)
for result in search_result.results:
    print(result.result_id)
    instance_id = result.result_id

https://bbp-nexus.epfl.ch/staging/v0/data/org1/experiment/subject/v0.1.0/09cb4bae-933d-4277-b24b-87b831beb1c0
https://bbp-nexus.epfl.ch/staging/v0/data/org1/experiment/subject/v0.1.0/a673c759-7a8b-48c4-b8f2-223f93bf764c
https://bbp-nexus.epfl.ch/staging/v0/data/org1/experiment/subject/v0.1.0/a69fd61f-fef9-448d-9b7d-f3a12f62284a


### 5-6 Update one of the instances

In [67]:
instance.data = updated_subject_data
client.instances.update(entity=instance)

### 5-7 Attach binaries to the dataset instances

### 5-8 Find your resources in Nexus Explorer, remove acls's, find all neurons